# 0. Install Dependencies lala

In [1]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym 
import random

In [3]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
actions

2

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:26.0
Episode:2 Score:15.0
Episode:3 Score:105.0
Episode:4 Score:22.0
Episode:5 Score:22.0
Episode:6 Score:12.0
Episode:7 Score:11.0
Episode:8 Score:19.0
Episode:9 Score:11.0
Episode:10 Score:21.0


# 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
import tensorflow
layers = tensorflow.keras.layers
BatchNormalization = tensorflow.keras.layers.BatchNormalization
Conv2D = tensorflow.keras.layers.Conv2D
Flatten = tensorflow.keras.layers.Flatten
TensorBoard = tensorflow.keras.callbacks.TensorBoard
ModelCheckpoint = tensorflow.keras.callbacks.ModelCheckpoint

#dikasih ini krn tadi error kecampur antara tensorflow dgn keras 
#https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

In [22]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [23]:
model = build_model(states, actions)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_6 (Dense)             (None, 24)                120       
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [30]:
from rl.agents import DQNAgent #agent yg digunakan buat ngetest 
from rl.policy import BoltzmannQPolicy #nge set policy
from rl.memory import SequentialMemory #maintain memory

In [34]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)  #ngeset policy memoru dwb trus di return dqn
    return dqn

In [35]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: 1.0000
25 episodes - episode_reward: 391.560 [37.000, 500.000] - loss: 11.029 - mae: 55.211 - mean_q: 110.902

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 60s 6ms/step - reward: 1.0000
24 episodes - episode_reward: 410.208 [117.000, 500.000] - loss: 10.194 - mae: 51.784 - mean_q: 104.378

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: 1.0000
22 episodes - episode_reward: 470.364 [190.000, 500.000] - loss: 10.450 - mae: 51.572 - mean_q: 103.994

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: 1.0000
21 episodes - episode_reward: 463.571 [217.000, 500.000] - loss: 10.724 - mae: 50.813 - mean_q: 102.320

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: 1.0

In [36]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 264.000, steps: 264
Episode 2: reward: 256.000, steps: 256
Episode 3: reward: 260.000, steps: 260
Episode 4: reward: 250.000, steps: 250
Episode 5: reward: 254.000, steps: 254
Episode 6: reward: 264.000, steps: 264
Episode 7: reward: 260.000, steps: 260
Episode 8: reward: 258.000, steps: 258
Episode 9: reward: 258.000, steps: 258
Episode 10: reward: 260.000, steps: 260
Episode 11: reward: 260.000, steps: 260
Episode 12: reward: 258.000, steps: 258
Episode 13: reward: 262.000, steps: 262
Episode 14: reward: 270.000, steps: 270
Episode 15: reward: 238.000, steps: 238
Episode 16: reward: 268.000, steps: 268
Episode 17: reward: 251.000, steps: 251
Episode 18: reward: 262.000, steps: 262
Episode 19: reward: 256.000, steps: 256
Episode 20: reward: 254.000, steps: 254
Episode 21: reward: 250.000, steps: 250
Episode 22: reward: 262.000, steps: 262
Episode 23: reward: 262.000, steps: 262
Episode 24: reward: 264.000, steps: 264
Episode 25: reward: 

In [37]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 272.000, steps: 272
Episode 2: reward: 262.000, steps: 262
Episode 3: reward: 240.000, steps: 240
Episode 4: reward: 266.000, steps: 266
Episode 5: reward: 262.000, steps: 262
Episode 6: reward: 242.000, steps: 242
Episode 7: reward: 260.000, steps: 260
Episode 8: reward: 247.000, steps: 247
Episode 9: reward: 254.000, steps: 254
Episode 10: reward: 248.000, steps: 248
Episode 11: reward: 240.000, steps: 240
Episode 12: reward: 262.000, steps: 262
Episode 13: reward: 260.000, steps: 260
Episode 14: reward: 260.000, steps: 260
Episode 15: reward: 260.000, steps: 260


# 4. Reloading Agent from Memory

In [44]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [45]:
del model
del dqn
del env

In [46]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [47]:
dqn.load_weights('dqn_weights.h5f')

In [48]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 240.000, steps: 240
Episode 2: reward: 258.000, steps: 258
Episode 3: reward: 260.000, steps: 260
Episode 4: reward: 246.000, steps: 246
Episode 5: reward: 248.000, steps: 248
